# YouTube 逐字稿分析器

這個程式可以：
1. 接收 YouTube URL 輸入
2. 提取影片逐字稿
3. 根據語言進行翻譯或加標點符號
4. 生成摘要

**注意：請確保影片長度在 4-5 分鐘以內，以避免記憶體不足的問題。**

In [ ]:
# 安裝必要套件
!pip install yt-dlp
!pip install openai-whisper
!pip install langchain-community
!pip install langchain

# 安裝 FFmpeg（用於音訊處理）
!apt update &> /dev/null
!apt install ffmpeg &> /dev/null

In [ ]:
# 安裝和設定 Ollama
!curl -fsSL https://ollama.com/install.sh | sh

# 啟動 Ollama 服務
import subprocess
import time

# 在背景啟動 Ollama 服務
process = subprocess.Popen(['ollama', 'serve'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(10)  # 等待服務啟動

# 下載 Gemma 7B 模型
!ollama pull gemma:7b

In [ ]:
# 匯入必要的套件
import os
import re
import yt_dlp
import whisper
import torch
from langchain_community.llms import Ollama
import warnings
warnings.filterwarnings("ignore")

print("套件匯入完成！")

In [ ]:
# 初始化模型
print("正在載入 Whisper 模型...")
whisper_model = whisper.load_model("base")

print("正在連接 LLM...")
try:
    llm = Ollama(model="gemma:7b", base_url="http://localhost:11434")
    # 測試連接
    test_response = llm.invoke("Hello")
    print("LLM 連接成功！")
except Exception as e:
    print(f"LLM 連接失敗: {e}")
    print("請確保 Ollama 服務正在運行")

In [ ]:
# 工具函數
def validate_youtube_url(url):
    """驗證 YouTube URL"""
    youtube_regex = re.compile(
        r'(https?://)?(www\.)?(youtube|youtu|youtube-nocookie)\.(com|be)/'
        r'(watch\?v=|embed/|v/|.+\?v=)?([^&=%\?]{11})'
    )
    return youtube_regex.match(url) is not None

def download_audio(url):
    """下載 YouTube 影片音訊"""
    print("正在下載影片音訊...")
    
    output_path = "/tmp/audio.%(ext)s"
    
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': output_path,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
    }
    
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        
        audio_file = "/tmp/audio.wav"
        if os.path.exists(audio_file):
            print("音訊下載完成！")
            return audio_file
        else:
            raise FileNotFoundError("音訊檔案未找到")
            
    except Exception as e:
        print(f"下載失敗: {e}")
        return None

def extract_transcript(audio_file):
    """使用 Whisper 提取逐字稿"""
    print("正在提取逐字稿...")
    
    try:
        result = whisper_model.transcribe(audio_file)
        transcript = result["text"]
        detected_language = result["language"]
        
        print(f"檢測到的語言: {detected_language}")
        print("逐字稿提取完成！")
        
        return transcript, detected_language
        
    except Exception as e:
        print(f"逐字稿提取失敗: {e}")
        return None, None

def detect_language(text):
    """檢測文本語言"""
    chinese_chars = len(re.findall(r'[\u4e00-\u9fff]', text))
    english_chars = len(re.findall(r'[a-zA-Z]', text))
    
    if chinese_chars > english_chars:
        return "zh"
    else:
        return "en"

def process_transcript_with_llm(transcript, language):
    """使用 LLM 處理逐字稿"""
    print(f"正在使用 LLM 處理逐字稿（語言: {language}）...")
    
    if language == "en":
        prompt = f"""請將以下英文逐字稿翻譯成繁體中文，保持原意並使用自然的中文表達：

{transcript}

請直接提供翻譯結果，不需要額外說明："""
    else:
        prompt = f"""請為以下中文逐字稿加上適當的標點符號，使其更容易閱讀：

{transcript}

請直接提供加上標點符號後的結果，不需要額外說明："""
    
    try:
        processed_text = llm.invoke(prompt)
        return processed_text.strip()
    except Exception as e:
        print(f"LLM 處理失敗: {e}")
        return transcript

def generate_summary(text):
    """生成摘要"""
    print("正在生成摘要...")
    
    prompt = f"""請將以下文本整理成條列式摘要，用繁體中文回答：

{text}

請用以下格式整理成重點摘要：
• 重點一
• 重點二
• 重點三
（依內容長度調整重點數量）

請直接提供摘要，不需要額外說明："""
    
    try:
        summary = llm.invoke(prompt)
        return summary.strip()
    except Exception as e:
        print(f"摘要生成失敗: {e}")
        return "摘要生成失敗"

print("工具函數定義完成！")

In [ ]:
# 主要執行程式
print("=== YouTube 逐字稿分析器 ===")
print()

# 1. 獲取 YouTube URL
url = input("請輸入 YouTube 影片 URL: ").strip()
print(f"您輸入的 URL: {url}")
print()

if not validate_youtube_url(url):
    print("URL 格式可能不正確，但仍嘗試處理...")

# 2. 下載音訊
audio_file = download_audio(url)
if not audio_file:
    print("音訊下載失敗，程式結束")
else:
    try:
        # 3. 提取逐字稿
        transcript, detected_language = extract_transcript(audio_file)
        if not transcript:
            print("逐字稿提取失敗，程式結束")
        else:
            # 4. 顯示原始逐字稿
            print("\n" + "="*50)
            print("原始逐字稿:")
            print("="*50)
            print(transcript)
            print()
            
            # 5. 使用 LLM 處理逐字稿
            language = detected_language if detected_language else detect_language(transcript)
            processed_transcript = process_transcript_with_llm(transcript, language)
            
            print("\n" + "="*50)
            if language == "en":
                print("翻譯結果:")
            else:
                print("加標點符號後的結果:")
            print("="*50)
            print(processed_transcript)
            print()
            
            # 6. 生成摘要
            summary = generate_summary(processed_transcript)
            
            print("\n" + "="*50)
            print("摘要:")
            print("="*50)
            print(summary)
            print()
            
    finally:
        # 7. 清理臨時檔案
        try:
            if audio_file and os.path.exists(audio_file):
                os.remove(audio_file)
                print("臨時檔案已清理")
        except Exception as e:
            print(f"清理臨時檔案時發生錯誤: {e}")
    
    print("程式執行完成！")